In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import string
from nltk.corpus import stopwords 
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
import emoji

In [2]:
df = pd.read_csv('../mi-person/data/go_emotions.csv')

In [3]:
df.shape
df.sample(7000)

,id,text,example_very_unclear,admiration,amusement,anger,annoyance,approval,caring,confusion,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
207674,eetgibp,"BT In the commentary booth: ""Where's your god ...",False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
105463,ee6w1dq,They'll make them walk the plank. Granny will ...,False,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
33124,eeeec7k,That's how it be sometimes.,True,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
95846,edu2qc9,Looks like your sources were closer to the act...,False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
107394,ed3792u,I feel this!,False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103805,ed1wkw8,Article is good. Not sure how it relates to “c...,False,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
29508,ef2n7oz,The only thing wrong with her is her face. May...,False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
66059,edffuj3,This team is fucking embarrassing.,False,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
126535,eeseie5,Hes looking for a handout of 3 free meals a da...,False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [4]:
df1 = df[df['amusement'] == 1]
df1.shape

(9245, 31)

In [5]:
df2 = df.drop(columns=['id', 'text', 'example_very_unclear'])
df_concat = pd.DataFrame()
for column in df2.columns:
    df_temp = df[df[column] == 1].sample(673)
    
    df_concat = pd.concat([df_concat, df_temp])

In [6]:
df_concat.shape

(18844, 31)

In [7]:
X_train = df_concat[['text']]
X_train.head()

,text
9826,I had to buy all your music after seeing that ...
92911,Good job ignoring [NAME].
150621,That would be very cool and a nice non-HP bar ...
199404,I love Caribbean food! I'm adding this one to ...
75627,[NAME] looking like [NAME] out there


In [8]:

def __remove_punctuation(text):
    """
        remove punctuation from text and lower case it
    """
    text = str(text)

    punctuations = string.punctuation
    punctuations += '“'
    punctuations += '’'
    punctuations += '”'
    punctuations += '’'
    punctuations += ' — '
    punctuations += 'â€œ'
    punctuations += 'â€¦'
    punctuations += 'â€'
    punctuations += '€™'
    punctuations += '€'
    punctuations += '™'
    punctuations += '¦'
    punctuations += 'œ'
    punctuations += 'Â'
    punctuations += 'Ã'
    punctuations += '— '
    punctuations += '¶'
    punctuations += '§'
    punctuations += '£'
    punctuations += '©'
    punctuations += 'ª'
    punctuations += '³'

    # text = emoji.get_emoji_regexp().sub(u'', text)

    for punctuation in punctuations:
        text = text.replace(punctuation, ' ') 
        #text = text.replace('donald', 'trump')
        #text = text.replace('clinton', 'hillary')
    return text.lower() # lower case

def __remove_numbers(text):
    """
        remove number from text
    """
    text = str(text)

    words_only = ''.join([i for i in text if not i.isdigit()])
    return words_only

def __remove_stopwords(text):
    """
        remove stop words from text
    """
    text = str(text)

    stop_words = stopwords.words('english')
    #stop_words += stopwords.words('portuguese')
    stop_words.append('mr')
    stop_words = set(stop_words)

    tokenized = word_tokenize(text)
    without_stopwords = [word for word in tokenized if not word in stop_words]
    return without_stopwords

def __lemmatize(text):
    """
        lemmatize text
    """
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(word) for word in text]
    lemmatized_string = " ".join(lemmatized)
    return lemmatized_string


def process_data(df):
    """
        process the data
    """

    df_ = df.copy()
        
    df_['text'] = df_['text'].apply(__remove_punctuation)

    df_['text'] = df_['text'].apply(__remove_numbers)

    df_['text'] = df_['text'].apply(__remove_stopwords)

    df_['text'] = df_['text'].apply(__lemmatize)
    
    return df_

In [9]:
cleaned_sentences = process_data(X_train).to_numpy()
cleaned_sentences

array([['buy music seeing vid amazing work'],
       ['good job ignoring name'],
       ['would cool nice non hp bar visualization'],
       ...,
       ['actually carrot'],
       ['ok dear'],
       ['one writer twitter really']], dtype=object)

In [12]:
# from nltk.corpus import stopwords
# from sklearn.feature_extraction.text import TfidfVectorizer

# tf_idf_vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'))
# processed_features = [tf_idf_vectorizer.fit_transform(cleaned_sentences) for sentence
#                       in cleaned_sentences]
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import multiprocessing


In [17]:
# weighted_words = pd.DataFrame(tf_idf_vectorizer.fit_transform(processed_features).toarray(),
#                  columns = tf_idf_vectorizer.get_feature_names_out())

# weighted_words
import re

def clean_data(text):
    text = re.sub(r'[^ \nA-Za-z0-9À-ÖØ-öø-ÿ/]+', '', text)
    text = re.sub(r'[\\/×\^\]\[÷]', '', text)
    return text

def change_lower(text):
    text = text.lower()
    return text

stopwords_list = stopwords.words("english")

def remover(text):
    text_tokens = text.split(" ")
    final_list = [word for word in text_tokens if not word in stopwords_list]
    text = ' '.join(final_list)
    return text

def get_w2vdf(df):
    w2v_df = pd.DataFrame(df["text"]).values.tolist()
    for i in range(len(w2v_df)):
        w2v_df[i] = w2v_df[i][0].split(" ")
    return w2v_df

def train_w2v(w2v_df):
    cores = multiprocessing.cpu_count()
    w2v_model = Word2Vec(min_count=4,
                         window=4,
                         alpha=0.03, 
                         min_alpha=0.0007, 
                         sg = 1,
                         workers=cores-1)
    
    w2v_model.build_vocab(w2v_df, progress_per=10000)
    w2v_model.train(w2v_df, total_examples=w2v_model.corpus_count, epochs=100, report_delay=1)
    return w2v_model

df[["text"]] = df[["text"]].astype(str)
df["text"] = df["text"].apply(change_lower)
df["text"] = df["text"].apply(clean_data)
df["text"] = df["text"].apply(remover)

w2v_df = get_w2vdf(df)
w2v_model = train_w2v(w2v_df)

In [24]:
w2v_df
w2v_model.wv.most_similar(positive=["great"])

[('zzzzzz', 0.5652974247932434),
 ('masked', 0.5458847880363464),
 ('hentaipoon', 0.5437642335891724),
 ('awesome', 0.5395288467407227),
 ('2cb', 0.5388690829277039),
 ('amazing', 0.5319364070892334),
 ('belgrade', 0.5256741642951965),
 ('margarine', 0.524250328540802),
 ('good', 0.5152955055236816),
 ('glutenfrei', 0.5046489238739014)]

In [25]:
w2v_model.wv.most_similar(positive=["terrible"])

[('incontinence', 0.628472626209259),
 ('debater', 0.6074535250663757),
 ('favouritism', 0.5826535224914551),
 ('teamlemoncreams', 0.5520726442337036),
 ('aggregate', 0.5432706475257874),
 ('plohamas', 0.5418367385864258),
 ('materials', 0.5376135110855103),
 ('moods', 0.5249935984611511),
 ('exploration', 0.5169201493263245),
 ('ballparks', 0.5020651817321777)]

In [26]:
w2v_model.wv.most_similar(positive=["boston"])

[('10000th', 0.5764591693878174),
 ('alds', 0.5188714861869812),
 ('1682', 0.4755295515060425),
 ('virginia', 0.4706263840198517),
 ('fatigued', 0.470175176858902),
 ('carolina', 0.46961137652397156),
 ('location', 0.4668193757534027),
 ('av', 0.462760329246521),
 ('1980', 0.45806530117988586),
 ('harvard', 0.4501142203807831)]

In [28]:
w2v_model.wv.most_similar(positive=["dog"])

[('edmunds', 0.6769403219223022),
 ('uglier', 0.6532434821128845),
 ('plaw', 0.62839674949646),
 ('degend', 0.6231839060783386),
 ('kets', 0.5727165341377258),
 ('wuf', 0.5635632872581482),
 ('brandy', 0.5564343929290771),
 ('intersepts', 0.5503106713294983),
 ('crating', 0.545530378818512),
 ('undeniable', 0.5446431636810303)]

In [32]:
model = Word2Vec(w2v_df, min_count=1)
model

In [16]:
y_train = df_concat.drop(columns=['id', 'text', 'example_very_unclear'])
y_train.shape

(18844, 28)

In [35]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.multiclass import OneVsRestClassifier

In [36]:
# model = OneVsRestClassifier(LogisticRegression()).fit(processed_features, y_train)
# model = OneVsRestClassifier(LogisticRegression()).fit(processed_features, y_train)

In [ ]:
df['text'].iloc[2]


"You do right, if you don't care then fuck 'em!"

In [ ]:
processed_test = vectorizer.transform(df['text'].values)
prediction = model.predict(processed_test)
prediction_proba = model.predict_proba(processed_test)

NameError: name 'model' is not defined

In [ ]:
#print(prediction[0])
print(prediction_proba[0].sum())
print(prediction_proba[0][np.argmax(prediction[2])]*100)
np.argmax(prediction[2])

1.5879333545366263
4.139329719007467


2

In [ ]:
processed_features.shape

(1, 1)

In [ ]:
# data = {"text": ["that game hurt"]}


In [ ]:
# X_test = pd.DataFrame.from_dict(data)
# test_vec = vectorizer.transform(X_test['text'].values)


In [ ]:
model.predict(processed_features)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
model.predict_proba(processed_features)

array([[0.44739292, 0.03586595, 0.02809953, ..., 0.02414154, 0.02709744,
        0.03835104],
       [0.058321  , 0.24651841, 0.06306535, ..., 0.05902671, 0.02587264,
        0.04027202],
       [0.10469759, 0.0209913 , 0.05397462, ..., 0.03384042, 0.02801978,
        0.04331961],
       ...,
       [0.04203796, 0.04283444, 0.0555304 , ..., 0.03986793, 0.01540153,
        0.09163799],
       [0.10061908, 0.03004642, 0.04698681, ..., 0.02987135, 0.01896844,
        0.0306477 ],
       [0.04314211, 0.03290709, 0.04596256, ..., 0.0680064 , 0.04222355,
        0.11853829]])

In [ ]:
model.score(processed_features, y_train)*100

7.99724050095521